In [1]:
import logging
import os
import sys

#spliter = '\\'
spliter = '/'

current_path = os.getcwd()
current_path_list = current_path.split(spliter)

for idx, file in enumerate(current_path_list):
    if file == 'src':
        break
sys.path.insert(0, spliter.join(current_path_list[:idx+1]))

In [2]:
current_path_list

['',
 'root',
 'autodl-tmp',
 'CMU-llms-11-667',
 'assignment3',
 'hw3-starter-code-2024.1.2',
 'src',
 'retriever',
 'driver']

In [3]:
from transformers import AutoTokenizer
from transformers import (
    HfArgumentParser,
    set_seed,
)

from retriever.arguments import ModelArguments, DataArguments, \
    TevatronTrainingArguments as TrainingArguments
from retriever.dataset import TrainDataset
from retriever.collator import TrainCollator
from retriever.modeling import EncoderModel as DenseModel
from retriever.trainer import TevatronTrainer as Trainer

In [4]:
logger = logging.getLogger(__name__)

model_to_train = 'jmvcoelho/pythia-160m-1024-marco-docs-bow-contrastive-pretrain'
trained_model_save_path = './data/model'

if not os.path.exists(trained_model_save_path):
    os.makedirs(trained_model_save_path)

trained_model_name = 'pythia-160m-1024-marco-docs-bow-contrastive-pretrain-marco-passage-sft'
training_data = 'Tevatron/msmarco-passage-aug'

In [6]:
parser = HfArgumentParser(
    (ModelArguments, DataArguments, TrainingArguments)
)

# sys.argv = [
#     "train.py",  # The script name (simulated)
#     "--model_name_or_path", "bert-base-uncased",
#     "--config_name", "bert-config.json",
#     "--dataset_path", "data/dataset.csv",
#     "--output_dir", "results/"
# ]

sys.argv = [
    "train.py",
      "--output_dir", f"{trained_model_save_path}/{trained_model_name}",
      "--model_name_or_path", model_to_train,
      "--dataset_name", training_data,
      "--save_steps", "0",
      "--temperature", "0.01",
      "--per_device_train_batch_size", "128",
      "--train_group_size", "10",
      "--learning_rate", "1e-4",
      "--query_max_len", "32",
      "--passage_max_len", "128",
      "--num_train_epochs", "1",
      "--logging_steps", "1",
      "--gradient_accumulation_steps", "2",
      "--run_name", trained_model_name
]

In [7]:
if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    model_args, data_args, training_args = parser.parse_json_file(
        json_file = os.path.abspath(sys.argv[1])
    )
    
else:
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    model_args: ModelArguments
    data_args: DataArguments
    training_args: TrainingArguments

In [8]:
if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

In [9]:
# Setup logging
logging.basicConfig(
    format = "%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt = "%m/%d/%Y %H:%M:%S",
    level = logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
)

logger.info("Training/evaluation parameters %s", training_args)
logger.info("MODEL parameters %s", model_args)

set_seed(training_args.seed)

01/02/2025 07:29:17 - INFO - __main__ -   Training/evaluation parameters TevatronTrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,

In [11]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', 
                        shell = True, 
                        capture_output = True, 
                        text = True
                       )
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [12]:
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir = model_args.cache_dir,
)

tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [13]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'right'

In [ ]:
model = DenseModel.build(
    model_args,
    training_args,
    cache_dir = model_args.cache_dir,
    attn_implementation = "flash_attention_2"
)

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/247M [00:00<?, ?B/s]

In [ ]:

    train_dataset = TrainDataset(data_args)
    collator = TrainCollator(data_args, tokenizer)

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = train_dataset,
        data_collator = collator
    )
    train_dataset.trainer = trainer

    trainer.train()
    trainer.save_model()
    if trainer.is_world_process_zero():
        tokenizer.save_pretrained(training_args.output_dir)
